# 使用 Keras 分析 TITANIC 乘客生还概率

In [132]:
# Imports
import numpy as np
import pandas as pd
import re
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

## 1. 加载数据


In [133]:
# Loading the data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_train['Age'].fillna(df_train['Age'].mean(), inplace=True)
df_train['Cabin'].fillna(0, inplace=True)
#df_train.fillna(0, inplace=True)
#df_test.fillna(0, inplace=True)

df_train.head()
#df_test.head()
#print(x_train.shape)
#print(x_test.shape)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S


In [134]:
df_train.info()
#df_train['Cabin'].map(lambda x: re.findall(r'[a-zA-Z]', str(x))
#title_unique = df_train['Name'].str.extract('([A-Za-z]+)\.', expand=False).unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [135]:
#cab2index = dict(zip(map(chr, range(ord('A'),ord('Z')+1)), [x+1 for x in range(26)]))
#df_train['SibSp'].unique()

In [136]:
df_train['title'] = df_train['Name'].str.extract('([A-Za-z]+)\.', expand=False)

title_unique = df_train['title'].unique()
tit2index = {y:x+1 for x,y in enumerate(title_unique)}

sex2index = {"male":0, "female":1}
emb2index = {"S":0, "C":1, "Q":2, "null":3}


x_total = df_train[['Pclass','SibSp','Parch']]
x_total['Sex'] = [sex2index[sex] for sex in df_train['Sex']]
x_total['Age'] = [int(age) for age in df_train['Age']]
x_total['Far'] = [int(far) for far in df_train['Fare']]
x_total['Cab'] = [1 if cab else 0 for cab in df_train['Cabin']]
x_total['Emb'] = [emb2index[emb] for emb in df_train['Embarked']]
#x_total['Tic'] = [re.findall('\d', ss) for ss in df_train['Ticket']]
x_total['Tic'] = [len(tic) for tic in df_train['Ticket']]
#x_total['Nam'] = [len(nam) for nam in df_train['Name']]
x_total['title'] = [tit2index[tit] for tit in df_train['title']]

y_total = df_train['Survived']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [137]:
x_total[:3]
#np.mean(df_train['Age'])

,Pclass,SibSp,Parch,Sex,Age,Far,Cab,Emb,Tic,title
0,3,1,0,0,22,7,0,0,9,1
1,1,1,0,1,38,71,1,1,8,2
2,3,0,0,1,26,7,0,0,16,3


In [138]:
x_train = np.array(x_total[:750])
y_train = np.array(y_total[:750])

x_test = np.array(x_total[750:])
y_test = np.array(y_total[750:])

x_train

array([[ 3,  1,  0, ...,  0,  9,  1],
       [ 1,  1,  0, ...,  1,  8,  2],
       [ 3,  0,  0, ...,  0, 16,  3],
       ...,
       [ 2,  0,  0, ...,  0,  6,  3],
       [ 1,  1,  0, ...,  0,  6,  1],
       [ 3,  0,  0, ...,  2,  6,  1]])

## 2. 检查数据



In [139]:
print(x_train[0])
print(y_train[0])

[ 3  1  0  0 22  7  0  0  9  1]
0


## 3. 检查数据集



In [140]:
# 
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(750, 2)
(141, 2)


## 4. 模型构建

使用 sequential 在这里构建模型。 添加 dropout 层以减少过拟合。

In [141]:
# Building the model architecture with one layer of length 100
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=10))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compiling the model using categorical_crossentropy loss, and rmsprop optimizer.
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 512)               5632      
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 1026      
Total params: 6,658
Trainable params: 6,658
Non-trainable params: 0
_________________________________________________________________


## 5. 训练模型

运行模型。 尝试不同的 batch_size 和 epoch ！

In [144]:
# Running and evaluating the model
hist = model.fit(x_train, y_train,
          batch_size=32,
          epochs=20,
          validation_data=(x_test, y_test), 
          verbose=2)

Train on 750 samples, validate on 141 samples
Epoch 1/20
 - 0s - loss: 0.6676 - acc: 0.7613 - val_loss: 0.7093 - val_acc: 0.7518
Epoch 2/20
 - 0s - loss: 0.6564 - acc: 0.7560 - val_loss: 0.4066 - val_acc: 0.8227
Epoch 3/20
 - 0s - loss: 0.7300 - acc: 0.7533 - val_loss: 0.4853 - val_acc: 0.7872
Epoch 4/20
 - 0s - loss: 0.6739 - acc: 0.7493 - val_loss: 1.3464 - val_acc: 0.6596
Epoch 5/20
 - 0s - loss: 0.7195 - acc: 0.7387 - val_loss: 0.4191 - val_acc: 0.7801
Epoch 6/20
 - 0s - loss: 0.7301 - acc: 0.7440 - val_loss: 0.6749 - val_acc: 0.7234
Epoch 7/20
 - 0s - loss: 0.6547 - acc: 0.7573 - val_loss: 0.4528 - val_acc: 0.8369
Epoch 8/20
 - 0s - loss: 0.7158 - acc: 0.7360 - val_loss: 0.3947 - val_acc: 0.8369
Epoch 9/20
 - 0s - loss: 0.6692 - acc: 0.7413 - val_loss: 0.5420 - val_acc: 0.7305
Epoch 10/20
 - 0s - loss: 0.7268 - acc: 0.7173 - val_loss: 0.3506 - val_acc: 0.8794
Epoch 11/20
 - 0s - loss: 0.6874 - acc: 0.7533 - val_loss: 0.3489 - val_acc: 0.8582
Epoch 12/20
 - 0s - loss: 0.7528 - acc:

## 6. 评估模型

准确率大于 85% 

In [146]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.8652482273730826
